In [14]:
import cv2
import pickle

# You can change the dimensions of the selection box for lips here
width, height = 145, 70

#Make a file to store the lip position
try:
    with open('MakeupPos', 'rb') as f:
        posList = pickle.load(f)
except:
    posList = []

def mouseClick(events, x, y, flags, params):
    if events == cv2.EVENT_LBUTTONDOWN:
        posList.append((x, y))
    if events == cv2.EVENT_RBUTTONDOWN:
        for i, pos in enumerate(posList):
            x1, y1 = pos
            if x1 < x < x1 + width and y1 < y < y1 + height:
                posList.pop(i)

    # Writing the selected space in the file for further use
    with open('MakeupPos', 'wb') as f:
        pickle.dump(posList, f)


while True:
    # Screenshot taken from the video
    img = cv2.imread('makeup1.png')
    for pos in posList:
        cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), (255, 0, 255), 2)

    cv2.imshow("Image", img)
    cv2.setMouseCallback("Image", mouseClick)

    # Check for the 'q' key press to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


In [15]:
import cv2
import pickle
import numpy as np

width, height = 145, 70

# Make a file to store the position
try:
    with open('MakeupPos', 'rb') as f:
        posList = pickle.load(f)
except:
    posList = []

# Initialize the lip color
lip_color = (0, 0, 0)

# Load the input image
image_path = 'makeup1.png'  # Replace with the actual path to your image
image = cv2.imread(image_path)

# Convert the image to the HSV color space
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Constants for lip color change
min_contour_area = 500  # Minimum contour area to consider as lips

# Lip color range in HSV
lower_lip_color = np.array([0, 50, 50])
upper_lip_color = np.array([10, 255, 255])

# Function to change the lip color
def change_color(x):
    global lip_color
    lip_color = (cv2.getTrackbarPos('B', 'Lip Color Changer'),
                 cv2.getTrackbarPos('G', 'Lip Color Changer'),
                 cv2.getTrackbarPos('R', 'Lip Color Changer'))
    change_lip_color()

# Function to apply the selected lip color
def change_lip_color():
    global image, hsv_image, mask
    # Apply the color thresholding to isolate the lips within the selected areas
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    for pos in posList:
        x1, y1 = pos
        x2, y2 = x1 + width, y1 + height
        mask[y1:y2, x1:x2] = cv2.inRange(hsv_image[y1:y2, x1:x2], lower_lip_color, upper_lip_color)

    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Filter contours based on area
    filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > min_contour_area]
    # Iterate over the filtered contours and draw filled lips with the selected color
    for contour in filtered_contours:
        cv2.drawContours(image, [contour], -1, lip_color, -1)
    # Display the result
    cv2.imshow('Lip Color Changer', image)

# Create a named window
cv2.namedWindow('Lip Color Changer')

# Create trackbars for color selection
cv2.createTrackbar('B', 'Lip Color Changer', 0, 255, change_color)
cv2.createTrackbar('G', 'Lip Color Changer', 0, 255, change_color)
cv2.createTrackbar('R', 'Lip Color Changer', 0, 255, change_color)

while True:
    # Display the image
    cv2.imshow('Lip Color Changer', image)

    # Check for the 'q' key press to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
